In [1]:
#ivory - 'https://www.ivory.co.il/%D7%98%D7%90%D7%91%D7%9C%D7%98%D7%99%D7%9D.html'
#  zap - 'https://www.zap.co.il/models.aspx?sog=c-tabletpc'
#  ksp - 'https://ksp.co.il/web/cat/270..1045'
# bug - 'https://www.bug.co.il/tablets/'
# מחסני חשמל- 'https://www.payngo.co.il/computers-pcs/smartphones/tablets.html?p=4'
# wisebuy- 'https://www.wisebuy.co.il/products.aspx?category=c-tabletpc'
#go mobile - 'https://www.gomobile.co.il/25137-%D7%98%D7%90%D7%91%D7%9C%D7%98%D7%99%D7%9D'
from bs4 import BeautifulSoup
import sqlite3
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

# def getIvoryData(url,_rowNum):
#   page=requests.get(url)
#   soup= BeautifulSoup(page.content,'html.parser')
#   cols=[]
#   val=[]
#   product=soup.findAll('li',attrs={'class':'col-md-12 col-12'})
#   for prod in product:
#     cols.append(prod.find('b').get_text())
#     val.append(prod.find('div',style='vertical-align: middle; width:59%; display:inline-block; padding-top: 5px; padding-bottom: 5px; color: #000000;').text.strip())
#   df=pd.DataFrame(columns=cols,index=range(1))
#   for value in range(len(cols)):
#     df.iloc[_rowNum][value]=val[value]
#   print(df)


# def getIvoryData(url):
#     page=requests.get(url)
#     soup= BeautifulSoup(page.content,'html.parser')
#     cols=[]
#     val=[]
#     product=soup.findAll('li',attrs={'class':'col-md-12 col-12'})
#     for prod in product:
#         cols.append(prod.find('b').get_text())
#         val.append(prod.find('div',style='vertical-align: middle; width:59%; display:inline-block; padding-top: 5px; padding-bottom: 5px; color: #000000;').text.strip())
# #        df=pd.DataFrame(columns=cols,index=range(1))
# #    for value in range(len(cols)):
# #        df.loc[_rowNum][value]=val[value]
# #    newdf=pd.concat([olddf,df])
#     return val



# url='https://www.ivory.co.il/catalog.php?act=cat&id=2592'
# page=requests.get(url)
# soup= BeautifulSoup(page.content,'html.parser')
# links=soup.find_all('a',class_='row product-anchor')
# count=0
# for link in links:   
#       df=getIvoryData(link['href'],count)
#       count+=1
# getIvoryData(links[0]['href'],0)




In [2]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 250))
def getIvoryHeaders():
    url='https://www.ivory.co.il/catalog.php?id=55718'
    page=requests.get(url)
    soup= BeautifulSoup(page.content,'html.parser')
    Header=[]
    product=soup.findAll('li',attrs={'class':'col-md-12 col-12'})
    for prod in product:
         Header.append(prod.find('div',style='vertical-align: middle; width:40%; display:inline-block; padding-top: 5px; padding-bottom: 5px;color: #000000;').text.strip())
    return Header

In [3]:
def getIvoryData(url,headers):
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
    product=soup.findAll('li',class_='col-md-12 col-12')
    pair=[]
    data={}
    for prod in product:
         temp=prod.find('b').text
         valtemp=prod.find('div',style='vertical-align: middle; width:59%; display:inline-block; padding-top: 5px; padding-bottom: 5px; color: #000000;').text.strip()
         if(temp in headers):
            data.update({temp:valtemp})
         else:
            data[temp]=None
    return data

In [4]:
url='https://www.ivory.co.il/catalog.php?act=cat&id=2592'
page=requests.get(url)
soup= BeautifulSoup(page.content,'html.parser')
links=soup.find_all('a',class_='row product-anchor')
count=0
column=getIvoryHeaders()
data={}
data=getIvoryData(links[0]['href'],column)
# for link in links:
df=pd.DataFrame(data.values(),columns=getIvoryHeaders())
df

ValueError: Shape of passed values is (20, 1), indices imply (20, 20)